In [1]:
import re
import gc
import os
import pickle
import random
from preprocess import preprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.model_selection import KFold,StratifiedKFold, train_test_split
import tensorflow as tf
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report,f1_score
from tensorflow.keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D, GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth',-1)

In [2]:
df = pd.read_csv(r'DK.csv', encoding='utf-8')

In [3]:
df['NegetiveFeedback'].value_counts()

0    2624635
1    308112 
Name: NegetiveFeedback, dtype: int64

In [4]:
df['len'] = df['Comment'].astype(str).apply(lambda x: len(x.split()))
df.head()

,Unnamed: 0,BadComment,Comment,Date,GoodComment,ItemURL,NegetiveFeedback,Person,PositiveFeedback,SubCatName,SubName,Unnamed: 0.1,Unnamed: 0.1.1,len
0,0,۰,<p>من کاملا با این محصول آشنا بودم و از خریدش مطمین بودم</p>,\r\n در تاریخ ۲ مهر ۱۳۹۹\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n سمانه خلج مهرآبادي\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,12
1,1,۲,<p>به درد خاصی نمیخوره صرفا یکی دو روز باهاش سرگرمین بعد به عنوان یه ساعت معمولی ازش استفاده میکنین</p>,\r\n در تاریخ ۱ مهر ۱۳۹۹\r\n,۴,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n محجوبه زارعی\r\n,0,ساعت هوشمند,کالای دیجیتال,NaN,NaN,19
2,2,۰,<p>در یک کلام به نسبت هزینه ای که میکنید کالمل ترین ساعت بازار رو دریافت می‌کنید<br/>\r\r\nدسترسی کامل به نوتیفیکشن ها و توانایی به پاسخ دادن به پیام ها از طریق خود ساعت<br/>\r\r\nهمچنین برنامه کامل ورزشی تمام نیاز های شما رو برآورده میکنه</p>,\r\n در تاریخ ۳۱ شهریور ۱۳۹۹\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n محمد رضا میرشکار سیاهکل\r\n,0,ساعت هوشمند,کالای دیجیتال,NaN,NaN,44
3,3,۰,<p>ساعت بسیار عالی و کاربردی هست،میخواستم برای دوستم هم بگیرم ولی به دلیل تفاوت قیمت زیاد از دیجی کالا نمیخرم،</p>,\r\n در تاریخ ۲۹ شهریور ۱۳۹۹\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n کاربر دیجی‌کالا\r\n,0,ساعت هوشمند,کالای دیجیتال,NaN,NaN,20
4,4,۰,<p>دقیقا مطابق آنچه انتظار می رفت بود .<br/>\r\r\nخوب و کاربردی و مطابق استایل روز .</p>,\r\n در تاریخ ۲۹ شهریور ۱۳۹۹\r\n,۱,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n محمد میرفخرایی\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,16


In [5]:
print(len(df))
df = df[(df['PositiveFeedback']==1)|(df['NegetiveFeedback']==1)]
print(len(df))

2932747
2057167


In [6]:
df['Label'] = 0
df.loc[df['NegetiveFeedback']==1,'Label'] = 1
df.head(25)

,Unnamed: 0,BadComment,Comment,Date,GoodComment,ItemURL,NegetiveFeedback,Person,PositiveFeedback,SubCatName,SubName,Unnamed: 0.1,Unnamed: 0.1.1,len,Label
0,0,۰,<p>من کاملا با این محصول آشنا بودم و از خریدش مطمین بودم</p>,\r\n در تاریخ ۲ مهر ۱۳۹۹\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n سمانه خلج مهرآبادي\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,12,0
4,4,۰,<p>دقیقا مطابق آنچه انتظار می رفت بود .<br/>\r\r\nخوب و کاربردی و مطابق استایل روز .</p>,\r\n در تاریخ ۲۹ شهریور ۱۳۹۹\r\n,۱,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n محمد میرفخرایی\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,16,0
5,5,۰,<p>خیییییییییییییییلی خوبه من تقریبا 8 ماهه دارمش خیلی خوبه</p>,\r\n در تاریخ ۲۸ شهریور ۱۳۹۹\r\n,۰,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n مریم کیال\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,9,0
6,6,۱,<p>بسیار عالی خدایی کار باهاش بسیار راحته . فقط قیمتش رو کمتر شه بهتره یکی دیگه باید بگیرم</p>,\r\n در تاریخ ۲۷ شهریور ۱۳۹۹\r\n,۲,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n وحید عارفی کیان\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,18,0
8,8,۲,<p>بسیار ساده و شیک و سبک است و کار کردن با آن آسان است</p>,\r\n در تاریخ ۲۶ شهریور ۱۳۹۹\r\n,۳,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n علیا شریعتی\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,14,0
11,11,۷,<p>مواردی که در نقاط قوت نوشتم، حداقل های یک واچ هستش. متاسفانه من متوجه شدم که وویس ها رو در واتس اپ نمیشه اصلاً گوش داد. فقط شما متوجه میشید که برای شما وویس اومده. برای پاسخ دادن به پیامها هم عملاً ناتوان هستید مگر اینکه بخواید جواب رو به انگلیسی بنویسید چون کیبورد که نداره و تایپ صوتی هم از زبان فارسی ساپورت نمیکنه. پس عملاً فقط متوجه تیتر نوتیفیکیشن ها خواهید شد. <br/>\r\r\nالکتروکاردیوگرام هم فعلاً بلااستفاده س. چون سازمان بهداشت جهانی هنوز این دستگاه رو تایید نکرده. کلاً خواستم بگم که همون ساعت های معمولی رو بخرید بهتر هم هستن.</p>,\r\n در تاریخ ۲۴ شهریور ۱۳۹۹\r\n,۴,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,1,\r\n علیرضا دریایی\r\n,0,ساعت هوشمند,کالای دیجیتال,NaN,NaN,103,1
13,13,۲,<p>کیفیت بسیار عالی بود</p>,\r\n در تاریخ ۲۲ شهریور ۱۳۹۹\r\n,۱,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n عبدالحسین توحیدی\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,4,0
15,15,۰,<p>من مدل ۴۰ صفحه نمایش را خریده بودم.پس از تماس با دیجی کالا مدل ۴۴ را جایگزین کردم که باید بگویم همکاری خوبی با من کردند.بازگشت وجه به سرعت انجام شد.<br/>\r\r\nبا تشکر از برخورد خوب پرسنل دیجی کالا<br/>\r\r\nبه اقایان حتما مدل ۴۴ را پیشنهاد میدهم</p>,\r\n در تاریخ ۲۱ شهریور ۱۳۹۹\r\n,۲,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n کاوه مشیری\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,47,0
16,16,۰,<p>قابلیت اندازه گیری فشار خون فعلا فقط توی کره جنوبی فعال شده و اواخر سال ۲۰۲۰ توی بقیه کشورها عمومی میشه</p>,\r\n در تاریخ ۲۳ شهریور ۱۳۹۹\r\n,۲,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n مهدی خدایی\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,21,0
18,18,۲,<p>با توجه به اینکه یکهفته بدستم رسیده هنوز نتوانستم سنسور فشار خون و کنترل دوربین از طریق ساعت هوشمند راه اندازی کنم <br/>\r\r\nنه دی جی کالا ونه خدمات سامسونگ هیچکدام راهنمایی نکرده و بعد این مدت به اندازه یه ساعت یک چهارم قیمت برایم کارایی دارد</p>,\r\n در تاریخ ۲۸ شهریور ۱۳۹۹\r\n,۳,/product/dkp-1878144/ساعت-هوشمند-سامسونگ-مدل-galaxy-watch-active2-44mm,0,\r\n محمدحسین رضائی\r\n,1,ساعت هوشمند,کالای دیجیتال,NaN,NaN,47,0


In [7]:
df['Label'].value_counts()

0    1749055
1    308112 
Name: Label, dtype: int64

In [9]:
prep = preprocess()
df['Comment'] = df['Comment'].apply(lambda x: prep.clean(x))

In [10]:
X = df['Comment'].values
Y = df['Label'].values

In [54]:
embed_num_dims = 100
max_seq_len = 400
EMB_SIZE = 100
MAX_LEN = 1000
seed = 2020
random.seed(seed)

skf = StratifiedKFold(n_splits=5)

batch_size = 2048
epoches = 8

In [55]:
tokenizer = Tokenizer(num_words=2000000,filters='!"#$%&()*+,-./;<=>?@][\\]^{|}~\t\n')
tokenizer.fit_on_texts(X)
index_of_words = tokenizer.word_index

In [56]:
def get_coefs(word,*arr):
    return word,np.asarray(arr,dtype='float32')
def build_embed_matrix(embed_path,word_index):
    EMB_SIZE =100
    embed_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embed_path, encoding='utf-8'))
    embed_matrix = np.zeros((len(word_index)+1,EMB_SIZE))
    
    for word , i in word_index.items():
        embed_vector = embed_index.get(word)
        if embed_vector is not None:
            embed_matrix[i]=embed_vector
    del embed_index
    gc.collect()
    return embed_matrix

In [57]:
embed_path = 'DigiKalaEmbeddingVevtors.vec'
embed_matrix_fasts = build_embed_matrix(embed_path,index_of_words)

In [58]:
def Evaluation(Ytest, pred_class, pred_proba):
    
    auc = roc_auc_score(Ytest,pred_proba)
    fpr, tpr, threshold = roc_curve(Ytest, pred_proba)
    
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    print('AUC:', auc)
    print('f1_score:', f1_score(Ytest,pred_class))
    print('Confusion Matrix:\n', confusion_matrix(Ytest,pred_class))

In [59]:
def create_model():
    model = Sequential()
    model.add(Embedding(len(index_of_words)+1, 100, input_length=max_seq_len, weights=[embed_matrix_fasts]))
    model.add(Dropout(0.5))
    model.add(Conv1D(128,kernel_size=3,padding='same',activation='relu',strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
for fold, (train_index, test_index) in enumerate(skf.split(X,Y)):
    
    print('#Fold:', fold+1)
    
    Xtrain, Xtest = X[train_index], X[test_index]
    Ytrain, Ytest = Y[train_index], Y[test_index]
    
    print('Oversampling ...')
    Xtrain = np.append(Xtrain, Xtrain[Ytrain==1])
    Ytrain = np.append(Ytrain, Ytrain[Ytrain==1])
    
    print('Pad Sequencing ...')
    sequence_train = tokenizer.texts_to_sequences(Xtrain)
    sequence_test = tokenizer.texts_to_sequences(Xtest)
    padata_train = pad_sequences(sequence_train,maxlen=max_seq_len, padding='post')
    padata_test = pad_sequences(sequence_test,maxlen=max_seq_len, padding='post')
    
    print('Modeling ...')
    model = create_model()
    history= model.fit(padata_train, Ytrain, validation_data=(padata_test,Ytest),epochs=epoches, batch_size=batch_size,
                       verbose=1, shuffle=True)
    
    print('Evaluating ...')
    pred_class = model.predict_classes(padata_test)
    pred_proba = model.predict(padata_test)
    Evaluation(Ytest, pred_class, pred_proba)

#Fold: 1
Oversampling ...
Pad Sequencing ...


In [ ]:
# psudo labeling ...

In [ ]:
res = pd.DataFrame({'Comment':Xtest, 'Label':Ytest, 'predict':predclas[:,0], 'predict_proba':predprob[:,0]})
res.head(20)

In [ ]:
res[res.Label!=res.predict].iloc[:,:]